# 파이썬 크롤링 

# 1. 필요 라이브러리 로드

In [155]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import time
from urllib.request import urlretrieve
import os
# 에러 무시 코드 
import warnings
warnings.filterwarnings("ignore")

In [6]:
# 설치가 안된 경우 / 라이브러리 설치하기
! pip install BeautifulSoup4

# 무신사 랭킹 페이지의 랭킹 정보를 크롤링하여 엑셀로 저장하기.
- 1. 웹페이지에서 원하는 정보를 검사하여 해당 데이터를 가져오기.
- 2. 가져온 데이터를 데이터 프레임화 시켜 엑셀 파일로 저장하기.

## 1. 해당 URL 페이지에서 원하는 정보를 검사하여 어떤 태그 및 클래스에 속해있는지 확인.

![alt text](image/image01.jpg "Title")

In [11]:
# 해당 URL 페이지의 정보를 가져와서 soup 변수에 저장해둔다. 
webpage = requests.get("https://search.musinsa.com/ranking/keyword")
soup = BeautifulSoup(webpage.content, "html.parser")

![alt text](image/image02.jpg "Title")

In [21]:
# 검색어 랭킹의 경우 div태그의 class가 search_inner인것을 확인하고 이를 find 함수를 통해 찾을 수 있다.
soup.find("div","search_inner")

# 앞으로 계속 사용할 변수임으로 변수명을 지정해주자.
ranking_search = soup.find("div","search_inner")

In [22]:
# ranking_search에 어떤 정보가 있는지 확인
ranking_search

<div class="search_inner">
<div class="tbl_box_sranking">
<ol class="sranking_list">
<li>
<a href="/search/musinsa/integration?q=%EC%88%8F%ED%8C%A8%EB%94%A9&amp;type=popular" title="숏패딩">
<p class="p_srank">001. 숏패딩</p>
<p class="p_srank_last">
<span class="arrow">-</span>
</p>
</a>
</li>
<li>
<a href="/search/musinsa/integration?q=%ED%9B%84%EB%93%9C%ED%8B%B0&amp;type=popular" title="후드티">
<p class="p_srank">002. 후드티</p>
<p class="p_srank_last">
<span class="arrow">-</span>
</p>
</a>
</li>
<li>
<a href="/search/musinsa/integration?q=%ED%8C%A8%EB%94%A9&amp;type=popular" title="패딩">
<p class="p_srank">003. 패딩</p>
<p class="p_srank_last">
<span class="arrow">-</span>
</p>
</a>
</li>
<li>
<a href="/search/musinsa/integration?q=%EB%A7%A8%ED%88%AC%EB%A7%A8&amp;type=popular" title="맨투맨">
<p class="p_srank">004. 맨투맨</p>
<p class="p_srank_last">
<span class="arrow">-</span>
</p>
</a>
</li>
<li>
<a href="/search/musinsa/integration?q=%EB%8B%88%ED%8A%B8&amp;type=popular" title="니트">
<p class="p_s

In [24]:
# ranking_search안에서 다시 검색어는 p 태크에 들어있는 것을 볼 수 있다. 
# 해당 관련 모든 태그는 find_all()을 이용하자.
ranking_search.find("p","p_srank")

<p class="p_srank">001. 숏패딩</p>

In [25]:
ranking_search.find_all("p","p_srank")

[<p class="p_srank">001. 숏패딩</p>,
 <p class="p_srank">002. 후드티</p>,
 <p class="p_srank">003. 패딩</p>,
 <p class="p_srank">004. 맨투맨</p>,
 <p class="p_srank">005. 니트</p>,
 <p class="p_srank">006. 후리스</p>,
 <p class="p_srank">007. 비니</p>,
 <p class="p_srank">008. 롱패딩</p>,
 <p class="p_srank">009. 코트</p>,
 <p class="p_srank">010. 경량패딩</p>,
 <p class="p_srank">011. 목도리</p>,
 <p class="p_srank">012. 드로우핏</p>,
 <p class="p_srank">013. 조거팬츠</p>,
 <p class="p_srank">014. 드로우핏 코트</p>,
 <p class="p_srank">015. 머플러</p>,
 <p class="p_srank">016. 후드집업</p>,
 <p class="p_srank">017. 예일</p>,
 <p class="p_srank">018. 엘무드</p>,
 <p class="p_srank">019. 나이키</p>,
 <p class="p_srank">020. 쿠어</p>,
 <p class="p_srank">021. 무탠다드</p>,
 <p class="p_srank">022. 커버낫</p>,
 <p class="p_srank">023. 양말</p>,
 <p class="p_srank">024. 무스탕</p>,
 <p class="p_srank">025. 플리스</p>,
 <p class="p_srank">026. 모자</p>,
 <p class="p_srank">027. 가디건</p>,
 <p class="p_srank">028. 발마칸 코트</p>,
 <p class="p_srank">029. 슬랙스</p>,
 <p class=

## 2. 찾은 정보를 데이터 프레임으로 만들기.

In [27]:
# 정보를 저장할 떄 태그가 붙어있을 필요가 없으니, text를 이용해 해당 부분의 text만 추출하자
ranking_search.find_all("p","p_srank")

[<p class="p_srank">001. 숏패딩</p>,
 <p class="p_srank">002. 후드티</p>,
 <p class="p_srank">003. 패딩</p>,
 <p class="p_srank">004. 맨투맨</p>,
 <p class="p_srank">005. 니트</p>,
 <p class="p_srank">006. 후리스</p>,
 <p class="p_srank">007. 비니</p>,
 <p class="p_srank">008. 롱패딩</p>,
 <p class="p_srank">009. 코트</p>,
 <p class="p_srank">010. 경량패딩</p>,
 <p class="p_srank">011. 목도리</p>,
 <p class="p_srank">012. 드로우핏</p>,
 <p class="p_srank">013. 조거팬츠</p>,
 <p class="p_srank">014. 드로우핏 코트</p>,
 <p class="p_srank">015. 머플러</p>,
 <p class="p_srank">016. 후드집업</p>,
 <p class="p_srank">017. 예일</p>,
 <p class="p_srank">018. 엘무드</p>,
 <p class="p_srank">019. 나이키</p>,
 <p class="p_srank">020. 쿠어</p>,
 <p class="p_srank">021. 무탠다드</p>,
 <p class="p_srank">022. 커버낫</p>,
 <p class="p_srank">023. 양말</p>,
 <p class="p_srank">024. 무스탕</p>,
 <p class="p_srank">025. 플리스</p>,
 <p class="p_srank">026. 모자</p>,
 <p class="p_srank">027. 가디건</p>,
 <p class="p_srank">028. 발마칸 코트</p>,
 <p class="p_srank">029. 슬랙스</p>,
 <p class=

## 실습1: 
- ranking_search.find_all("p","p_srank")를 새로운 변수에 담기.
- ranking_search.find_all("p","p_srank")를 통해 구한 검색어 결과의 길이를 확인해보기.
- ranking_search.find_all("p","p_srank")의 길이를 100개 까지만 추출해보기.

In [ ]:
# ranking_search.find_all("p","p_srank")를 새로운 변수에 담기.


In [ ]:
# ranking_search.find_all("p","p_srank")를 통해 구한 검색어 결과의 길이를 확인해보기.


In [ ]:
# ranking_search.find_all("p","p_srank")의 길이를 100개 까지만 추출해보기.


## 태그로 얻어진 결과에서 Text만 추출하기.

In [40]:
# 해당 데이터는 태그와 클래스 정보를 모두 가지고 있다.
ranking_search.find_all("p","p_srank")[0]

<p class="p_srank">001. 숏패딩</p>

In [283]:
ranking_search.find_all("p","p_srank")[0]['class']

['p_srank']

In [41]:
# .text를 통해서 해당 데이터의 text부분만 가져올 수 있다. 
ranking_search.find_all("p","p_srank")[0].text

'001. 숏패딩'

## 실습 2:
- for문을 활용해 ranking_search.find_all("p","p_srank")에서 100개를 text 부분만 추출하기.
- for문을 활용해 ranking_search.find_all("p","p_srank")에서 100개를 text 부분만 추출하여 list에 추가하기.
- 해당 리스트를 데이터 프레임으로 만들기.
- 데이터 프레임을 CSV 파일로 저장하기.


In [58]:
# for문을 활용해 ranking_search.find_all("p","p_srank")에서 100개를 text 부분만 추출하기.


In [54]:
# for문을 활용해 ranking_search.find_all("p","p_srank")에서 100개를 text 부분만 추출하여 list에 추가하기.


In [57]:
# 해당 리스트를 데이터 프레임으로 만들기.


In [ ]:
# 데이터 프레임을 CSV 파일로 저장하기.


# 최종 실습

## 무신사 뉴스 페이지에서 뉴스 헤드라인 저장하기
https://magazine.musinsa.com/?m=news

## 워크 플로우
- 1. 검사를 이용하여 뉴스 헤드라인 데이터 추출하기.
- 2. For 문을 활용하여 현재 페이지 전체의 데이터 추출하기.
- 3. Url을 이동하여 그 다음 페이지 전체의 데이터 추출하기.


In [314]:
# 첫번째 페이지의 첫번째 제목 불러오기 
# 실행예제:


'인플루언서들이 선택한 잔스포츠 인기템은?'

In [307]:
# 페이지 전체의 제목 불러오기
# 실행 예제:


글로벌 스타 정호연의 매력, 캘빈클라인 진 21 F/W 화보 공개
스우파 멤버들이 입은 데님 셋업은? 살롱드서울 신상품 발매
프로스펙스가 알려주는 시티보이 코디 팁
대세는 파스텔 컬러 숏 패딩! 스파오 우신사 랭킹 1위
인사일런스의 모던한 감각을 닮은 더현대 서울 팝업 스토어
원 마일 웨어의 넥스트 레벨, 폴로 스포츠 신상품 발매
11월 1주차 우신사 주간 랭킹 : 시선 사로잡는 초겨울 인기 아이템 10
클라이밍 무드 담은 예일 × 와일드띵스 협업 컬렉션 한정 발매
귀여운 스누피를 만난 라코스테 × 피너츠 컬렉션
플리스 특가 기회! 무신사 스탠다드 무신사 라이브 예정
겨울에도 고프코어, 그라미치 최대 40% 할인
대세 로고와 함께하는 마크 곤잘레스 ‘슈무’ 캠페인
감각적인 무드 더한 디스커버리 익스페디션 스트릿 컬렉션
귀여운 댕댕이와 커플 룩! 엘엠씨 퍼피&프렌즈 컬렉션 한정 발매
예고된 1위! 디스이즈네버댓 × 고어텍스 인피니엄
스타일도 기능성도 확실한 만남! 이벳필드 × 디미토 한정 발매
필드와 일상에서 모두 즐기자! 스트레치엔젤스 프리미엄 숏 패딩 컬렉션
모던 클래식템 득템 기회! 헤지스 할인전 오픈
포근한 솜사탕 같은 질바이질스튜어트 21 윈터 컬렉션
포근함 극강! 앤더슨벨 포 우먼 테디베어 코트 1위


In [309]:
# 첫번째부터 10번째 페이지까지 제목 불러오기
# 실행예제:


무신사에서만 만날 수 있는 살로몬 스포츠스타일 신상품은?
매력 폭발 컬러 포인트! 반스 스타일 36 재입고
모두가 기다린 디스이즈네버댓 × 고어텍스 인피니엄 한정 발매
11월 1주차 무신사 주간 랭킹 : 찬 바람 철통방어 하는 보온 꿀템 톱 10
인플루언서가 선택한 커버낫 대세 아우터는?
지리산 상큼 레인저 고민시가 네파 팝업에 방문한 이유는?
무신사 × 오징어 게임 인기 질주! 초록색 체육복 랭킹 1위
무신사와 깐부 맺은 ′오징어 게임′, 초록색 체육복 단독 발매
차정원이 입은 마리떼를 직접 눈으로 확인하는 방법은?
키르시 체리 컬러 담은 아식스 젤 벤쳐 6 1위
키치&빈티지 무드, 키르시 × 아식스 한정 발매
포근한 스웨터에 상큼한 컬러 더한 아메스 월드와이드 신상품
강렬한 컬러로 매력을 뽐내는 K뷰티 1등템 3CE 입점
방 분위기 확 바꾸는 팔렛 × 러브조이 한정 발매
최대 50% 할인, 가죽 명가 도프제이슨의 무신사 라이브 이벤트홀
절제된 무드와 정교한 실루엣, 드로우핏 21 F/W 컬렉션 한정 발매
11월 2주차 발매되는 놓치지 말아야 할 신상품 리스트
10월 4주차 스니커즈 랭킹 : 빈티지가 대세! 인기 스니커즈 랭킹
지금 가장 입기 좋은 핸드메이드 코트! 무신사 스탠다드 우먼즈 인기
무신사 스탠다드 우먼즈 신상품으로 겨울 아우터 걱정 끝내자
추위 막는 무신사 스탠다드 신상품 발매
더현대 서울에서 만나는 알파 인더스트리 팝업 스토어
키르시 체리 컬러로 꾸민 아식스 스니커즈의 발매일은?
엄브로 플라이트 숏 다운 입은 스우파 아이키 화보 공개
친환경 충전재 담은 필루미네이트 21 윈터 컬렉션 한정 발매
위트 넘치는 디테일 가득, 미스터차일드 21 윈터 컬렉션 한정 발매
축구에 진심인 FC구척장신은 골스튜디오를 입는다
블랙야크 입고 아이유, 카이 포토카드 받자!
박신혜도 반한 까스텔바작 톤온톤 필드 룩
트렌드 이끄는 질스튜어트 스포츠 뉴욕 롱 다운 1위
10월 4주차 플리스 랭킹 : 포근해서 더 좋은 인기 플리스
10월 4주차 우신사 롱 패딩 랭킹 : 인

# Selenium을 활용한 동적 페이지 크롤링

In [262]:
# 웹드라이브로 크롬브라우즈 띄운다.
driver_path = "driver/chromedriver.exe"
driver = webdriver.Chrome(executable_path=driver_path)

In [197]:
url_page = "https://store.musinsa.com/app/styles/lists"
driver.get(url_page)
html = driver.page_source
soup = BeautifulSoup(html, "lxml")

# 코디 페이지에 있는 코디 이미지 저장해보기

![alt text](image/image03.jpg "Title")

In [147]:
images = driver.find_elements_by_css_selector(".style-list-thumbnail__img")[1].get_attribute("src")

In [148]:
images

'https://image.msscdn.net/images/style/list/l_3_2021110914273600000095308.jpg'

In [159]:
img_folder_path = os.getcwd()+'/image'  
 
if not os.path.isdir(img_folder_path):    
    os.mkdir(img_folder_path)
    
img_list = []
for i in range(len(driver.find_elements_by_css_selector(".style-list-thumbnail__img"))):
    img_list.append(driver.find_elements_by_css_selector(".style-list-thumbnail__img")[i].get_attribute("src"))
    
for index, link in enumerate(img_list) :
    urlretrieve(link, img_folder_path+'/'+str(index)+'.jpg')

# 마우스 이벤트 활용하기.

In [166]:
# 스크롤 내리기
driver.execute_script("window.scrollTo(0, 2000)")

In [173]:
# 다음페이지 클릭하기 
driver.find_element_by_xpath("/html/body/div[3]/div[3]/form/div[4]/div/div[4]/div/div/a[13]").click()